In [1]:
import os
import sys
sys.path.append("../")
sys.path.append("../src")

import lmdb
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
from PIL import Image

import caffe
from experiment import top_30_class as experiment
from mit_data import MITDataset

/home/nakamura/pytorch-i3d


In [2]:
dataset = MITDataset(split_file=experiment.SPLIT_FILE, index_file=experiment.INDEX_FILE)
index = dataset.index
mlb = dataset.mlb
bin_labels = dataset.binary_label

In [3]:
def get_video_file(direc, filename):
    return os.path.join(direc, filename)

make pair of (image, binarized-label) for whole dataset.

image should in (C x W x H) format and BGR

In [4]:
map_size = np.zeros((3, 256, 256), dtype=np.uint8).nbytes * 5000000
num_item = len(dataset) * 90
print(num_item)
random_idx = random.sample(range(100000), 100000)

59130


In [5]:
env = lmdb.open('../data/MIT_data/video_lmdb', map_size=map_size)
idx = 0

with env.begin(write=True) as txn:
    # txn is a Transaction object
    for i in range(len(dataset)):
        video_path = get_video_file(index[i]["directory"], index[i]["filename"])
        video = dataset.load_video(video_path)[:, :, :, ::-1]
        video = video.transpose([0, 3, 1, 2])
        labels = bin_labels[i]
        for frame in range(len(video)):
            frame = video[frame]
            
            datum = caffe.proto.caffe_pb2.Datum()
            datum.channels = frame.shape[0]
            datum.height = frame.shape[1]
            datum.width = frame.shape[2]
            datum.data = frame.tobytes()  # or .tostring() if numpy < 1.9
            str_id = '{:08}'.format(random_idx[idx])
            idx += 1
            txn.put(str_id, datum.SerializeToString())

In [6]:
env = lmdb.open('../data/MIT_data/label_lmdb', map_size=map_size)
idx = 0

with env.begin(write=True) as txn:
    # txn is a Transaction object
    for i in range(len(dataset)):
        video_path = get_video_file(index[i]["directory"], index[i]["filename"])
        video = dataset.load_video(video_path)[:, :, :, ::-1]
        video = video.transpose([0, 3, 1, 2])
        labels = bin_labels[i].reshape([-1,1,1])
        for frame in range(len(video)):
            dat = caffe.io.array_to_datum(labels)
            str_id = '{:08}'.format(random_idx[idx])
            idx += 1
            txn.put(str_id, dat.SerializeToString())

In [8]:
dataset = experiment.val_dataset
index = dataset.index
mlb = dataset.mlb
bin_labels = dataset.binary_label

In [9]:
env = lmdb.open('../data/MIT_data/video_val_lmdb', map_size=map_size)
idx = 0

with env.begin(write=True) as txn:
    # txn is a Transaction object
    for i in range(len(dataset)):
        video_path = get_video_file(index[i]["directory"], index[i]["filename"])
        video = dataset.load_video(video_path)[:, :, :, ::-1]
        video = video.transpose([0, 3, 1, 2])
        labels = bin_labels[i]
        for frame in range(len(video)):
            frame = video[frame]
            
            datum = caffe.proto.caffe_pb2.Datum()
            datum.channels = frame.shape[0]
            datum.height = frame.shape[1]
            datum.width = frame.shape[2]
            datum.data = frame.tobytes()  # or .tostring() if numpy < 1.9
            str_id = '{:08}'.format(idx)
            idx += 1
            txn.put(str_id, datum.SerializeToString())

In [10]:
env = lmdb.open('../data/MIT_data/label_val_lmdb', map_size=map_size)
idx = 0

with env.begin(write=True) as txn:
    # txn is a Transaction object
    for i in range(len(dataset)):
        video_path = get_video_file(index[i]["directory"], index[i]["filename"])
        video = dataset.load_video(video_path)[:, :, :, ::-1]
        video = video.transpose([0, 3, 1, 2])
        labels = bin_labels[i].reshape([-1,1,1])
        for frame in range(len(video)):
            dat = caffe.io.array_to_datum(labels)
            str_id = '{:08}'.format(idx)
            idx += 1
            txn.put(str_id, dat.SerializeToString())